In [ ]:
# !pip install fastparquet

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
import pandas as pd
import numpy as np
import datetime as dt
import warnings

# 2019-Oct-Ver2

In [ ]:
df = pd.read_parquet('/content/drive/MyDrive/MyCP2/2019-Oct.parquet')

In [ ]:
df['event_type'] = df['event_type'].astype('category')
df['category_code'] = df['category_code'].astype('category')

In [ ]:
df1['category_code'] = df1['category_code'].astype('string')
df1.update(df1['category_code'].fillna('NaN.NaN.NaN'))
df1['category_code'] = df1['category_code'].astype('category')

In [ ]:
df1['main_category'] = df1['category_code'].str.split('.', expand = True)[0].astype('category')
df1['sub_category'] = df1['category_code'].str.split('.', expand = True)[1].astype('category')
df1['sub_sub_category'] = df1['category_code'].str.split('.', expand = True)[2].astype('category')

In [ ]:
df1 = df1.drop(columns = ['category_code'])

In [ ]:
df1['brand'] = df1['brand'].astype('string')
df1.update(df1['brand'].fillna('unknown'))
df1['brand'] = df1['brand'].astype('category')

In [ ]:
df1['purchased'] = np.where(df1['event_type'] == 'purchase' , 1, 0)
df1['viewed'] = np.where(df1['event_type'] == 'view' , 1, 0)
df1['put_cart'] = np.where(df1['event_type'] == 'cart' , 1, 0)

In [ ]:
df2 = df1.astype({'product_id' : 'category', 'category_id' : 'category', 'user_id' : 'category', 'user_session' : 'category'})

In [ ]:
df2.to_parquet('/content/drive/MyDrive/MyCP2/2019-Oct-Ver2.parquet')

# ItemsUsers

In [ ]:
path = '/content/drive/MyDrive/MyCP2/2019-Oct-Ver2.parquet'
data = pd.read_parquet(path, engine='fastparquet')

df = data.copy()

In [ ]:
df.columns

Index(['event_time', 'event_type', 'product_id', 'category_id', 'brand',
       'price', 'user_id', 'user_session', 'main_category', 'sub_category',
       'sub_sub_category', 'purchased'],
      dtype='object')

In [ ]:
# users_id_num 컬럼 생성
df_users = pd.DataFrame(df['user_id'].drop_duplicates())
df_users = df_users.assign(
    users_id_num=np.arange(len(df_users))
    ).reset_index(drop=True)

# items_id_num 컬럼 생성
df_items = pd.DataFrame(df['product_id'].drop_duplicates())
df_items = df_items.assign(
    items_id_num=np.arange(len(df_items))
    ).reset_index(drop=True)

In [ ]:
# id_num merge
df = pd.merge(df, df_users)
df = pd.merge(df, df_items)

In [ ]:
# items_users_id_tuple 컬럼 생성
df['items_users_id_tuple'] = list(
    zip(df.users_id_num, df.items_id_num)
    )

In [ ]:
# 'purchased', 'user_session' 컬럼 삭제
df = df.drop(['purchased', 'user_session'], axis=1)

In [ ]:
# ItemsUsers 파케이로 저장
df.to_parquet('/content/drive/MyDrive/MyCP2/ItemsUsers.parquet')

# Items

In [ ]:
path = '/content/drive/MyDrive/MyCP2/2019-Oct-Ver2.parquet'
data = pd.read_parquet(path, engine='fastparquet')

df = data.copy()

In [ ]:
# items_id_num 컬럼 생성
df_items = pd.DataFrame(df['product_id'].drop_duplicates())
df_items = df_items.assign(
    items_id_num=np.arange(len(df_items))
    ).reset_index(drop=True)

df = pd.merge(df, df_items)

In [ ]:
df['viewed'] = np.where(df['event_type'] == 'view' , 1, 0)
df['put_cart'] = np.where(df['event_type'] == 'cart' , 1, 0)

In [ ]:
df.columns

Index(['event_time', 'event_type', 'product_id', 'category_id', 'brand',
       'price', 'user_id', 'user_session', 'main_category', 'sub_category',
       'sub_sub_category', 'purchased', 'items_id_num', 'viewed', 'put_cart'],
      dtype='object')

In [ ]:
df1 = df[['product_id', 'user_id', 'main_category', 'sub_category', 'sub_sub_category', 'viewed', 'put_cart', 'purchased', 'items_id_num']]

In [ ]:
df1.head()

,product_id,user_id,main_category,sub_category,sub_sub_category,viewed,put_cart,purchased,items_id_num
0,6200670,516364071,appliances,environment,air_heater,1,0,0,0
1,6200670,516364071,appliances,environment,air_heater,1,0,0,0
2,6200670,513595110,appliances,environment,air_heater,1,0,0,0
3,6200670,520226582,appliances,environment,air_heater,1,0,0,0
4,6200670,520226582,appliances,environment,air_heater,1,0,0,0


In [ ]:
## product_id별 view_num, cart_num, purchase_num 컬럼 생성

# product_id 중복값 삭제
df2 = pd.DataFrame(df1['product_id'].drop_duplicates())

# product_id별 view, cart, purchas 횟수 계산
df3 = pd.DataFrame(df1.groupby(['product_id'])['viewed'].sum()).reset_index()
df4 = pd.DataFrame(df1.groupby(['product_id'])['put_cart'].sum()).reset_index()
df5 = pd.DataFrame(df1.groupby(['product_id'])['purchased'].sum()).reset_index()

df6 = pd.merge(df2, df3)
df7 = pd.merge(df6, df4)
df8 = pd.merge(df7, df5)

df8.rename(columns = {'viewed':'view_num', 'put_cart':'cart_num', 'purchased':'purchase_num'},inplace=True)

In [ ]:
df8.head()

,product_id,view_num,cart_num,purchase_num
0,6200670,279,0,2
1,13400451,112,0,0
2,2601131,113,1,2
3,12703494,33808,458,723
4,12710449,4337,0,59


In [ ]:
# product_id별 카테고리 컬럼 추가
df10 = df[['product_id', 'main_category', 'sub_category', 'sub_sub_category', 'items_id_num']]
df11 = df10.drop_duplicates()

df9 = pd.merge(df8, df11)

In [ ]:
df9.head()

,product_id,view_num,cart_num,purchase_num,main_category,sub_category,sub_sub_category,items_id_num
0,6200670,279,0,2,appliances,environment,air_heater,0
1,13400451,112,0,0,NaN,NaN,NaN,1
2,2601131,113,1,2,NaN,NaN,NaN,2
3,12703494,33808,458,723,NaN,NaN,NaN,3
4,12710449,4337,0,59,NaN,NaN,NaN,4


In [ ]:
# category 하나로 묶기
cols = ['main_category', 'sub_category', 'sub_sub_category']
df9['category'] = df9[cols].apply(lambda row: ','.join(row.values.astype(str)), axis=1)

In [ ]:
df9.drop(['main_category', 'sub_category', 'sub_sub_category'], axis=1, inplace=True)

In [ ]:
# 예시
df9[df9['category'].str.contains('electronics')]

,product_id,view_num,cart_num,purchase_num,items_id_num,category
5,4803795,1892,90,31,5,"electronics,audio,headphone"
7,4800236,2817,88,55,7,"electronics,audio,headphone"
9,1801537,2592,54,43,9,"electronics,video,tv"
14,1802010,3670,0,73,14,"electronics,video,tv"
25,5801695,539,0,5,25,"electronics,audio,subwoofer"
...,...,...,...,...,...,...
132265,22400259,1,0,0,132265,"electronics,audio,microphone"
132295,4803416,1,0,0,132295,"electronics,audio,headphone"
132302,5801739,1,0,0,132302,"electronics,audio,subwoofer"
132303,5801738,1,0,0,132303,"electronics,audio,subwoofer"


In [ ]:
# items_features 컬럼 생성: (items_id_num, [category])
features = df9[['category']].apply(
    lambda x: ','.join(x.map(str)), axis=1)
features = features.str.split(',')
features = list(zip(df9['items_id_num'], features))

df9['items_features'] = features

In [ ]:
df9

,product_id,view_num,cart_num,purchase_num,items_id_num,category,items_features
0,6200670,279,0,2,0,"appliances,environment,air_heater","(0, [appliances, environment, air_heater])"
1,13400451,112,0,0,1,"NaN,NaN,NaN","(1, [NaN, NaN, NaN])"
2,2601131,113,1,2,2,"NaN,NaN,NaN","(2, [NaN, NaN, NaN])"
3,12703494,33808,458,723,3,"NaN,NaN,NaN","(3, [NaN, NaN, NaN])"
4,12710449,4337,0,59,4,"NaN,NaN,NaN","(4, [NaN, NaN, NaN])"
...,...,...,...,...,...,...,...
132323,24400526,1,0,0,132323,"NaN,NaN,NaN","(132323, [NaN, NaN, NaN])"
132324,24400527,4,0,0,132324,"NaN,NaN,NaN","(132324, [NaN, NaN, NaN])"
132325,26020645,3,0,0,132325,"NaN,NaN,NaN","(132325, [NaN, NaN, NaN])"
132326,24400252,1,0,0,132326,"NaN,NaN,NaN","(132326, [NaN, NaN, NaN])"


In [ ]:
df9['items_features'] = df9['items_features'].astype(str)

In [ ]:
df9.to_parquet('/content/drive/MyDrive/MyCP2/Items.parquet')

In [ ]:
df9

,product_id,view_num,cart_num,purchase_num,items_id_num,category,items_features
0,6200670,279,0,2,0,"appliances,environment,air_heater","(0, ['appliances', 'environment', 'air_heater'])"
1,13400451,112,0,0,1,"NaN,NaN,NaN","(1, ['NaN', 'NaN', 'NaN'])"
2,2601131,113,1,2,2,"NaN,NaN,NaN","(2, ['NaN', 'NaN', 'NaN'])"
3,12703494,33808,458,723,3,"NaN,NaN,NaN","(3, ['NaN', 'NaN', 'NaN'])"
4,12710449,4337,0,59,4,"NaN,NaN,NaN","(4, ['NaN', 'NaN', 'NaN'])"
...,...,...,...,...,...,...,...
132323,24400526,1,0,0,132323,"NaN,NaN,NaN","(132323, ['NaN', 'NaN', 'NaN'])"
132324,24400527,4,0,0,132324,"NaN,NaN,NaN","(132324, ['NaN', 'NaN', 'NaN'])"
132325,26020645,3,0,0,132325,"NaN,NaN,NaN","(132325, ['NaN', 'NaN', 'NaN'])"
132326,24400252,1,0,0,132326,"NaN,NaN,NaN","(132326, ['NaN', 'NaN', 'NaN'])"


# Users

In [ ]:
path = '/content/drive/MyDrive/MyCP2/ItemsUsers.parquet'
data = pd.read_parquet(path, engine='fastparquet')

df = data.copy()

In [ ]:
df.columns

Index(['event_time', 'event_type', 'product_id', 'category_id', 'brand',
       'price', 'user_id', 'main_category', 'sub_category', 'sub_sub_category',
       'users_id_num', 'items_id_num', 'items_users_id_tuple'],
      dtype='object')

In [ ]:
df1 = df[['user_id', 'product_id', 'event_type']]

In [ ]:
# 같은 user의 같은 item에 대한 중복 view, cart, purchase 삭제
df1 = df1.drop_duplicates()

In [ ]:
len(df1)

14363483

In [ ]:
df1 = df1.groupby(
    ['user_id', 'product_id'])['event_type'].apply(
        ','.join).reset_index()

In [ ]:
df1.rename(columns = {'event_type':'events'},inplace=True)

In [ ]:
# 예시
df1[df1['events'].str.contains('cart')]

,user_id,product_id,events
762,340041246,9300037,"view,purchase,cart"
1200,371460797,3701016,"view,cart"
1337,373096692,2900570,"view,cart"
1338,373096692,2900888,"view,cart"
2076,393318901,5100503,"view,cart"
...,...,...,...
13924181,566232270,1801690,"view,cart,purchase"
13924230,566233096,5100816,"view,cart,purchase"
13924293,566235051,4804295,"view,cart"
13924358,566236063,3700790,"view,cart,purchase"


In [ ]:
df1.to_parquet('/content/drive/MyDrive/MyCP2/Users.parquet')

In [ ]:
path = '/content/drive/MyDrive/MyCP2/Users.parquet'
data = pd.read_parquet(path, engine='fastparquet')

df = data.copy()

In [ ]:
df1 = data.copy()

In [ ]:
df.head()

,user_id,product_id,events
0,64078358,10600284,view
1,183503497,22200103,view
2,184265397,6902133,view
3,184265397,6902303,view
4,184265397,27400002,view


In [ ]:
df1['events'].unique()

array(['view', 'view,purchase', 'view,purchase,cart', 'view,cart',
       'view,cart,purchase', 'cart,view', 'cart,view,purchase',
       'purchase,view', 'purchase', 'cart,purchase', 'cart',
       'cart,purchase,view', 'purchase,view,cart'], dtype=object)

In [ ]:
# users_id_num 컬럼 추가
df_users = pd.DataFrame(df['user_id'].drop_duplicates())
df_users = df_users.assign(
    users_id_num=np.arange(len(df_users))
    ).reset_index(drop=True)

df2 = pd.merge(df1, df_users)

In [ ]:
# Users_features 컬럼 생성: (users_id_num, [events])
features1 = df2[['events']].apply(
    lambda x: ','.join(x.map(str)), axis=1)
features1 = features1.str.split(',')
features1 = list(zip(df2['users_id_num'], features1))

df2['users_features'] = features1

In [ ]:
df2

,user_id,product_id,events,users_id_num,users_features
0,64078358,10600284,view,0,"(0, [view])"
1,183503497,22200103,view,1,"(1, [view])"
2,184265397,6902133,view,2,"(2, [view])"
3,184265397,6902303,view,2,"(2, [view])"
4,184265397,27400002,view,2,"(2, [view])"
...,...,...,...,...,...
13924729,566242771,6800317,view,2042287,"(2042287, [view])"
13924730,566242873,3601405,view,2042288,"(2042288, [view])"
13924731,566242875,3601405,view,2042289,"(2042289, [view])"
13924732,566242880,3601405,view,2042290,"(2042290, [view])"


In [ ]:
# 예시
df2[df2['user_id']==566233096]

,user_id,product_id,events,users_id_num,users_features
13924230,566233096,5100816,"view,cart,purchase",669801,"(669801, [view, cart, purchase])"


In [ ]:
df2['users_features'] = df2['users_features'].astype(str)

In [ ]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 13924734 entries, 0 to 13924733
Data columns (total 5 columns):
 #   Column          Dtype 
---  ------          ----- 
 0   user_id         int64 
 1   product_id      int64 
 2   events          object
 3   users_id_num    int64 
 4   users_features  object
dtypes: int64(3), object(2)
memory usage: 637.4+ MB


In [ ]:
df2.to_parquet('/content/drive/MyDrive/MyCP2/Users.parquet')

In [ ]:
df2

,user_id,product_id,events,users_id_num,users_features
0,64078358,10600284,view,0,"(0, ['view'])"
1,183503497,22200103,view,1,"(1, ['view'])"
2,184265397,6902133,view,2,"(2, ['view'])"
3,184265397,6902303,view,2,"(2, ['view'])"
4,184265397,27400002,view,2,"(2, ['view'])"
...,...,...,...,...,...
13924729,566242771,6800317,view,2042287,"(2042287, ['view'])"
13924730,566242873,3601405,view,2042288,"(2042288, ['view'])"
13924731,566242875,3601405,view,2042289,"(2042289, ['view'])"
13924732,566242880,3601405,view,2042290,"(2042290, ['view'])"


In [ ]:
df2[df2['user_id']==566233096]

,user_id,product_id,events,users_id_num,users_features
13924230,566233096,5100816,"view,cart,purchase",2042056,"(2042056, ['view', 'cart', 'purchase'])"
